In [7]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_csv('GSAF5.csv', encoding = 'latin1')

In [5]:
df.sample(frac=1).head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
3744,1960.08.22.R.,Reported 22-Aug-1960,1960,Unprovoked,PAPUA NEW GUINEA,Milne Bay Province,NaN,Free-diving,"""a native""",M,...,NaN,"The Age, 8/22/1960",1960.08.22.R-Milne-PNG.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1960.08.22.R.,1960.08.22.R.,2249,NaN,NaN
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4,NaN,NaN
4717,1934.02.24,24-Feb-34,1934,Unprovoked,AUSTRALIA,Torres Strait,Adolphus Channel,Splashing in water,"Rixon, an aboriginal",M,...,3.4 m [11'] tiger shark & a 5' shark,"V.M. Coppleson (1958), p.243",1934.02.24-Rixon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1934.02.24,1934.02.24,1276,NaN,NaN
2531,1988.01.05.R,Reported 05-Jan-1988,1988,Provoked,SOUTH AFRICA,Western Cape Province,Great Brak River,"Returning to shore, collided with shark","5 m inflatable boat, occupants: Kobus Potgiete...",NaN,...,6 m shark,"Natal Mercury, 1/5/1988",1988.01.05-Potgeiter-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1988.01.05.R,1988.01.05.R,3463,NaN,NaN
828,2010.01.12,12-Jan-10,2010,Unprovoked,SOUTH AFRICA,Western Province,Fish Hoek,Standing,Lloyd Skinner,M,...,White shark,"L. Cohen, Times Live, 1/12/2010",2010.01.12-Skinner.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2010.01.12,2010.01.12,5165,NaN,NaN


In [6]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [8]:
df['Date'].value_counts().sort_index()

    10-Jan-2009    1
    15-Jun-1937    1
    16-Jan-1970    1
    22-Jul-2013    1
   21-Sep-1908     1
                  ..
Summer-2008        1
Winter 1942        1
Winter 1969        1
Woirld War II      1
World War II       2
Name: Date, Length: 5128, dtype: int64

In [9]:
df.loc[0,'Date']

'18-Sep-16'

In [10]:
pd.to_datetime(df.loc[0,'Date'],infer_datetime_format = True)

Timestamp('2016-09-18 00:00:00')

In [11]:
def are_you_inferable(date):
    if pd.to_datetime(date, infer_datetime_format = True):
        return True
    else:
        False

In [12]:
are_you_inferable('18-Sep-16')

True

In [15]:
df['Date'].apply(are_you_inferable)

ValueError: ('Unknown string format:', 'Reported  14-Jul-2016')

This is not working

In [16]:
def are_you_inferable2(date):
    try:
        pd.to_datetime(date, infer_datetime_format = True)
        return True
    except:
        return False

In [17]:
are_you_inferable2('Reported  14-Jul-2016')

False

In [19]:
df['Date'].apply(are_you_inferable2)

0        True
1        True
2        True
3        True
4        True
        ...  
5987    False
5988    False
5989    False
5990    False
5991    False
Name: Date, Length: 5992, dtype: bool

In [20]:
df['Date'].apply(are_you_inferable2).value_counts()

True     5135
False     857
Name: Date, dtype: int64

Move the 857 rows that could not be infered into a separated df for further inspection

In [23]:
df_ninfer = df[df['Date'].apply(are_you_inferable2) == False]
df_ninfer.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
33,2016.07.14.4,Reported 14-Jul-2016,2016,Unprovoked,BAHAMAS,NaN,Tiger Beach,Scuba Diving,Michael Dornellas,M,...,"Lemon shark, 9'","GrindTV, 7/14/2016",2016.07.14.R-TigerBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.14.R,2016.07.14.4,5960,NaN,NaN
34,2016.07.08.R,Reported 08-Jul-2016,2016,Unprovoked,SPAIN,Canary Islands,"Las Teresitas, Tenerife",Wading,female,F,...,Angel shark,La Opinion de Tenerife,2016.07.08.R-Spain.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.08.R,2016.07.08.R,5959,NaN,NaN
85,2016.03.03.R,Reported 03-Mar-2016,2016,Unprovoked,AUSTRALIA,South Australia,Wrights Bay,Fishing,Lee Taplin,M,...,Bronze whaler,"9 News, 3/1/2016",2016.03.03.R-Taplin.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.03.R,2016.03.03.R,5908,NaN,NaN
90,2016.02.10.R,Reported 10-Feb-2016,2016,Invalid,CAYMAN ISLANDS,Grand Cayman,Stingray City Bar,Feeding stingrays?,Richard Branson,M,...,No shark involvement,R. Branson,2016.02.10.R-Branson-stingray.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.02.10.R,2016.02.10.R,5903,NaN,NaN
100,2016.01.11.R,Reported 11-Jan-2016,2016,Unprovoked,AUSTRALIA,Queensland,"Happy Valley Beach, Caloundra",Surfing,Shane Hilder,M,...,Wobbegong shark,"ABC Sunshine Coast, 1/11/2016",2016.01.11.R-Hilder.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.01.11.R,2016.01.11.R,5893,NaN,NaN


In [24]:
df_ninfer.shape

(857, 24)

In [25]:
df_ninfer['Date'].value_counts()

No date                 6
No date, Before 1963    5
1970s                   5
Before 1958             4
Reported 10-Oct-1906    4
                       ..
Reportd 15-Jul-1894     1
Reported 07-Mar-1930    1
Reported 06-Mar-1886    1
Reported 15-Dec-1909    1
1844.07.16.R            1
Name: Date, Length: 771, dtype: int64

In [30]:
date_pattern = r'(\d{4}\.\d{2}\.\d{2})'
df_ninfer['Case Number'].str.extract(date_pattern, expand = False)

33      2016.07.14
34      2016.07.08
85      2016.03.03
90      2016.02.10
100     2016.01.11
           ...    
5987           NaN
5988           NaN
5989           NaN
5990           NaN
5991           NaN
Name: Case Number, Length: 857, dtype: object

In [31]:
df_ninfer['Case Number'].str.extract(date_pattern, expand = False).apply(are_you_inferable2) == False

33      False
34      False
85      False
90      False
100     False
        ...  
5987    False
5988    False
5989    False
5990    False
5991    False
Name: Case Number, Length: 857, dtype: bool